# Fetch comments

In [ ]:
import os
import pandas as pd

# Convert the dictionary to a DataFrame
all_data = pd.DataFrame()
for i in range(1, 2094):
    file = f'../data/json/comment/page_{i}.json'
    data = pd.read_json(file, typ='series').to_dict()
    df = pd.DataFrame.from_dict(data, orient='index')
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'uid'}, inplace=True)
    all_data = pd.concat([all_data, df], ignore_index=True)
all_data.reset_index(drop=True, inplace=True)
all_data.to_parquet('../data/comments.parquet')


In [ ]:
comments = pd.read_parquet('../data/comments.parquet')
comments.head()


In [ ]:
users_with_comments = comments.copy()[comments['total'] > 0]
users_with_comments.rename(columns={'total': 'comments'}, inplace=True)
users_with_comments


In [ ]:
total_rows, _ = comments.shape
comment_rows, _ = users_with_comments.shape
comments_rate = '{:.2f}%'.format((comment_rows / total_rows * 100))
f"{comments_rate} of all user have ever made at least one comment on d.o ({comment_rows} of {total_rows})"


In [ ]:
# Merge comments with users
users = pd.read_parquet('../data/user.parquet')
users.rename(columns={'id': 'uid'}, inplace=True)
users.head()


In [ ]:
df = pd.merge(users_with_comments, users, on='uid', how='left')

# Remove the system_message bot user.
system_message_uid = 180064
df = df[df['uid'] != system_message_uid]
df


In [ ]:
df.groupby('region')['comments'].count()\
    .plot(kind='bar', color='blue', legend=False, title='Commentors per Region (at least 3 comments)')


In [ ]:
# Top commentors
top = df.nlargest(100, 'comments')
top
